# Learning Objectives

In this notebook, you will craft sophisticated ETL jobs that interface with a variety of common data sources, such as 
- REST APIs (HTTP endpoints)
- RDBMS
- Hive tables (managed tables)
- Various file formats (csv, json, parquet, etc.)


# Interview Questions

As you progress through the practice, attempt to answer the following questions:

## Columnar File
- What is a columnar file format and what advantages does it offer?
- Why is Parquet frequently used with Spark and how does it function?
- How do you read/write data from/to a Parquet file using a DataFrame?

## Partitions
- How do you save data to a file system by partitions? (Hint: Provide the code)
- How and why can partitions reduce query execution time? (Hint: Give an example)

## JDBC and RDBMS
- How do you load data from an RDBMS into Spark? (Hint: Discuss the steps and JDBC)

## REST API and HTTP Requests
- How can Spark be used to fetch data from a REST API? (Hint: Discuss making API requests)

## ETL Job One: Parquet file
### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Data transformation requirements https://pgexercises.com/questions/aggregates/fachoursbymonth.html

### Load
Load data into a parquet file

### What is Parquet? 

Columnar files are an important technique for optimizing Spark queries. Additionally, they are often tested in interviews.
- https://www.youtube.com/watch?v=KLFadWdomyI
- https://www.databricks.com/glossary/what-is-parquet

In [0]:
from pyspark.sql.functions import sum, col

bookings_df = spark.table("bookings")
members_df = spark.table("members")
facilities_df = spark.table("facilities")

result_df = (
    bookings_df.groupBy("facid")
    .agg(sum(col("slots")).alias("Total Slots"))
    .orderBy("facid") 
)

result_df.show()

output_path = "/data/slots"

result_df.write.parquet(output_path)




+-----+-----------+
|facid|Total Slots|
+-----+-----------+
|    0|       1320|
|    1|       1278|
|    2|       1209|
|    3|        830|
|    4|       1404|
|    5|        228|
|    6|       1104|
|    7|        908|
|    8|        911|
+-----+-----------+



## ETL Job Two: Partitions

### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Transform the data https://pgexercises.com/questions/joins/threejoin.html

### Load
Partition the result data by facility column and then save to `threejoin_delta` managed table. Additionally, they are often tested in interviews.

hint: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.partitionBy.html

What are paritions? 

Partitions are an important technique to optimize Spark queries
- https://www.youtube.com/watch?v=hvF7tY2-L3U&t=268s

In [0]:
from pyspark.sql.functions import concat_ws

result_df = (bookings_df.join(facilities_df, col("bookings.facid") == col("facilities.facid")).join(members_df, col("bookings.memid") == col("members.memid")))
result_df = result_df.filter(col("name").isin("Tennis Court 1", "Tennis Court 2")).select(
        col("name").alias("facility_name"), 
        concat_ws(" ", col("firstname"), col("surname")).alias("member_name")
    ).distinct().orderBy("member_name", "facility_name")

result_df.show()

output_path = "/data/facility"

result_df.write.partitionBy("facility_name").parquet(output_path)


+--------------+--------------+
| facility_name|   member_name|
+--------------+--------------+
|Tennis Court 1|    Anne Baker|
|Tennis Court 2|    Anne Baker|
|Tennis Court 1|  Burton Tracy|
|Tennis Court 2|  Burton Tracy|
|Tennis Court 1|  Charles Owen|
|Tennis Court 2|  Charles Owen|
|Tennis Court 2|  Darren Smith|
|Tennis Court 1| David Farrell|
|Tennis Court 2| David Farrell|
|Tennis Court 1|   David Jones|
|Tennis Court 2|   David Jones|
|Tennis Court 1|  David Pinker|
|Tennis Court 1| Douglas Jones|
|Tennis Court 1| Erica Crumpet|
|Tennis Court 1|Florence Bader|
|Tennis Court 2|Florence Bader|
|Tennis Court 1|   GUEST GUEST|
|Tennis Court 2|   GUEST GUEST|
|Tennis Court 1|Gerald Butters|
|Tennis Court 2|Gerald Butters|
+--------------+--------------+
only showing top 20 rows



## ETL Job Three: HTTP Requests

### Extract
Extract daily stock price data price from the following companies, Google, Apple, Microsoft, and Tesla. 

Data Source
- API: https://rapidapi.com/alphavantage/api/alpha-vantage
- Endpoint: GET `TIME_SERIES_DAILY`

Sample HTTP request

```
curl --request GET \
	--url 'https://alpha-vantage.p.rapidapi.com/query?function=TIME_SERIES_DAILY&symbol=TSLA&outputsize=compact&datatype=json' \
	--header 'X-RapidAPI-Host: alpha-vantage.p.rapidapi.com' \
	--header 'X-RapidAPI-Key: [YOUR_KEY]'

```

Sample Python HTTP request

```
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "function":"TIME_SERIES_DAILY",
    "symbol":"IBM",
    "datatype":"json",
    "outputsize":"compact"
}

headers = {
    "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
    "X-RapidAPI-Key": "[YOUR_KEY]"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# Now 'data' contains the daily time series data for "IBM"
```

### Transform
Find **weekly** max closing price for each company.

hints: 
  - Use a `for-loop` to get stock data for each company
  - Use the spark `union` operation to concat all data into one DF
  - create a new `week` column from the data column
  - use `group by` to calcualte max closing price

### Load
- Partition `DF` by company
- Load the DF in to a managed table called, `max_closing_price_weekly`

In [0]:
import requests
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

url = "https://alpha-vantage.p.rapidapi.com/query"
headers = {
    "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
    "X-RapidAPI-Key": "9f33b7490bmsh607e6ec0a839621p1c6ab9jsn91d5f9c379ad"
}

tickers = ["GOOG", "AAPL", "MSFT", "TSLA"]

schema = StructType([
    StructField("date", StringType(), True),
    StructField("company", StringType(), True),
    StructField("high", StringType(), True)
])

result_df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

for element in tickers:
    querystring = {
        "function": "TIME_SERIES_WEEKLY",
        "symbol": element,
        "datatype": "json",
        "outputsize": "compact"
    }

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    weekly_data = data['Weekly Time Series']

    data_list = []
    for date, values in weekly_data.items():
        data_list.append((date, element, values["2. high"])) 

    ticker_df = spark.createDataFrame(data_list, schema=schema) 
    result_df = result_df.union(ticker_df)

result_df = result_df.orderBy(col("date").desc())

# Save as a managed table with partitioning by company
result_df.write.mode("overwrite").partitionBy("company").saveAsTable("max_closing_price_weekly")


+----------+-------+--------+
|      date|company|    high|
+----------+-------+--------+
|2025-03-18|   GOOG|168.4600|
|2025-03-18|   TSLA|245.4000|
|2025-03-18|   MSFT|392.7050|
|2025-03-18|   AAPL|215.2200|
|2025-03-14|   TSLA|253.3700|
|2025-03-14|   MSFT|390.2300|
|2025-03-14|   GOOG|170.4500|
|2025-03-14|   AAPL|236.1600|
|2025-03-07|   TSLA|303.9400|
|2025-03-07|   MSFT|402.1500|
|2025-03-07|   GOOG|176.9000|
|2025-03-07|   AAPL|244.0272|
|2025-02-28|   TSLA|342.3973|
|2025-02-28|   MSFT|409.3700|
|2025-02-28|   GOOG|185.0900|
|2025-02-28|   AAPL|250.0000|
|2025-02-21|   TSLA|367.3400|
|2025-02-21|   MSFT|419.3100|
|2025-02-21|   GOOG|187.7800|
|2025-02-21|   AAPL|248.6900|
+----------+-------+--------+
only showing top 20 rows



In [0]:
spark.sql("SELECT * FROM max_closing_price_weekly").show()

+----------+-------+--------+
|      date|company|    high|
+----------+-------+--------+
|2025-03-18|   AAPL|215.2200|
|2025-03-14|   AAPL|236.1600|
|2025-03-07|   AAPL|244.0272|
|2025-02-28|   AAPL|250.0000|
|2025-02-21|   AAPL|248.6900|
|2025-02-14|   AAPL|245.5500|
|2025-02-07|   AAPL|234.0000|
|2025-01-31|   AAPL|247.1900|
|2025-01-24|   AAPL|227.0300|
|2025-01-17|   AAPL|238.9600|
|2025-01-10|   AAPL|247.3300|
|2025-01-03|   AAPL|253.5000|
|2024-12-27|   AAPL|260.1000|
|2024-12-20|   AAPL|255.0000|
|2024-12-13|   AAPL|250.8000|
|2024-12-06|   AAPL|244.6300|
|2024-11-29|   AAPL|237.8100|
|2024-11-22|   AAPL|230.7199|
|2024-11-15|   AAPL|228.8700|
|2024-11-08|   AAPL|228.6600|
+----------+-------+--------+
only showing top 20 rows



## ETL Job Four: RDBMS


### Extract
Extract RNA data from a public PostgreSQL database.

- https://rnacentral.org/help/public-database
- Extract 100 RNA records from the `rna` table (hint: use `limit` in your sql)
- hint: use `spark.read.jdbc` https://docs.databricks.com/external-data/jdbc.html

### Transform
We want to load the data as it so there is no transformation required.


### Load
Load the DF in to a managed table called, `rna_100_records`

In [0]:
Hostname =  "hh-pgsql-public.ebi.ac.uk"
Port = "5432"
Database = "pfmegrnargs"
User = "reader"
Password = "NWDMCE5xdipIjRrp"

employees_table = (spark.read
  .format("jdbc")
  .option("driver", "org.postgresql.Driver")
  .option("url", "jdbc:postgresql://" + Hostname + ":" + Port + "/" + Database)
  .option("dbtable", "rna")
  .option("user", User)
  .option("password", Password)
  .load()
)

employees_table = employees_table.limit(100)

employees_table.write.mode("overwrite").saveAsTable("rna_100_records")



In [0]:
spark.sql("SELECT * FROM rna_100_records").show()

+--------+-------------+--------------------+---------+----------------+----+--------------------+--------+--------------------+
|      id|          upi|           timestamp|userstamp|           crc64| len|           seq_short|seq_long|                 md5|
+--------+-------------+--------------------+---------+----------------+----+--------------------+--------+--------------------+
| 8988357|URS00008926C5| 2015-10-20 18:04:07|   RNACEN|F9626977AB4E17FB|1336|TCAGCGGCGAACGGGTG...|    null|fe4792a9218a34fde...|
| 8988360|URS00008926C8| 2015-10-20 18:04:07|   RNACEN|DEA611A8ABDE9078|1307|ACTGCTATCGGATTGAT...|    null|5eb946fc85a2e16f4...|
| 8988361|URS00008926C9| 2015-10-20 18:04:07|   RNACEN|AE161A21AF6713C0|1367|AGCCCAGCTTGCTGGGT...|    null|fe4849b1977b5be3c...|
| 8988362|URS00008926CA| 2015-10-20 18:04:07|   RNACEN|03DF15DE82E78D7F|1398|GAGTTTGATCATGGCTC...|    null|c4bb7b410de36a58c...|
| 8988364|URS00008926CC| 2015-10-20 18:04:07|   RNACEN|AE0439B061E1640E|1409|GTCGAACGGTAACAGGA...